In [1]:
import datajoint as dj
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import ipyvolume.pylab as p3

In [2]:
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')

Connecting cpapadop@10.28.0.34:3306


In [3]:
fetched_mesh = (ta3p100.Mesh & ta3p100.CurrentSegmentation & 'segment_id=648518346341351503').fetch1()

In [4]:
fetched_mesh = ta3p100.Decimation35.fetch(limit=1, as_dict=True)[0]

In [3]:
fetched_mesh = (ta3p100.Mesh & ta3p100.CurrentSegmentation & 'segment_id=648518346341366885').fetch1()

In [4]:
class Voxel:
    def __init__(self, vertices):
        self._vertices = vertices.copy()
        
    @property
    def vertices(self):
        return self._vertices
        
    @staticmethod
    def get_bbox(vertices):
        return np.array([(np.min(axis), np.max(axis)) for axis in vertices.T])
        
    @property
    def bbox(self):
        return self.get_bbox(self.vertices)
    
    @property
    def _rectangular_idx(self):
        X = [0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
        Y = [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0]
        Z = [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
        return np.vstack((X, Y, Z))
    
    @property
    def voxels(self):
        return self._voxels
    
    @voxels.setter
    def voxels(self, voxels):
        self._voxels
    
    @property
    def offset(self):
        return self._offset
    
    @offset.setter
    def offset(self, side_length):
        self._offset = side_length
    
    @property
    def structure(self):
        return self._structure
        
    @structure.setter
    def structure(self, structure):
        self._structure = structure # I just store each voxel as an offset value vector.
    
    # Could have a more configurable plotting function. Like you give the argument ['mesh', 'structure', 'mesh_bbox', etc.]
    def plot_structure(self, width=800, height=600):
        p3.figure(width=width, height=height)
#         p3.plot_trisurf(*mesh.vertices.T/1000, triangles=mesh.triangles)
        p3.plot_trisurf(*fetched_mesh['vertices'].T/1000, triangles=fetched_mesh['triangles'])
        initial_bottom = mesh.bbox[:,0]
        offset = self.offset
        rectangular_idx = self._rectangular_idx
        for i, vector in enumerate(self.structure):
            voxel_min = initial_bottom + (vector * offset)
            voxel_max = voxel_min + offset
            voxel_bbox = np.vstack((voxel_min, voxel_max)).T
            p3.plot(*np.array([axis[idx] for axis, idx in zip(voxel_bbox, rectangular_idx)])/1000, color='blue')
            if i > 1000:
                break
        p3.squarelim()
        p3.show()
    
#     @structure.indexer
#     def structure(self, index_tuple):
#         return self.structure[index_tuple] # self._structure[index_tuple]

#     def plot_voxels(self):
#         # Scale and transpose
#         center = [4, 4, 4];
#         cubesize = 2;
#         # Vertices for Line Cube. Order matters
#         X = [0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
#         Y = [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0]
#         Z = [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
#         # Example two cube matrix. Unit cube and one scaled/translated cube
#         X1 = [X X*cubesize+center(1)];
#         Y1 = [Y Y*cubesize+center(2)];
#         Z1 = [Z Z*cubesize+center(3)];
#         # Single plot command for all 'cube lines'
#         plot3(X1,Y1,Z1);
    
    def plot_mesh(self, width=800, height=600):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*fetched_mesh['vertices'].T/1000, triangles=fetched_mesh['triangles'])
        p3.plot(*np.array([axis[idx] for axis, idx in zip(self.bbox, self._rectangular_idx)])/1000, color='blue')
        p3.squarelim()
        p3.show()
    
    def plot_voxels(self, width=800, height=600):
        rectangular_idx = self._rectangular_idx
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*fetched_mesh['vertices'].T/1000, triangles=fetched_mesh['triangles'])
        [p3.plot(*np.array([axis[idx] for axis, idx in zip(bbox, rectangular_idx)])/1000, color='blue') for bbox in self.voxels]
        p3.squarelim()
        p3.show()
    
    @staticmethod
    def _sort_vertex_rows(vertices):
        # This still might be slower.
        a = vertices
        a = a[a[:,2].argsort()] # First sort doesn't need to be stable.
        a = a[a[:,1].argsort(kind='mergesort')]
        a = a[a[:,0].argsort(kind='mergesort')]
        return a
    
    def cube_voxelize(self, side_length):
        self.offset = side_length
        bbox = self.bbox
        # Get the number of voxels to be used to create cubes (allowing for pushing past the boundaries).
        num_voxels = np.ceil((np.abs(np.subtract(*bbox.T) / side_length))).astype(int)
        
        # Create the cube voxel grid split structure. Could also sort the vertices according to the grid at this point?
        start_coord = bbox.T[0]
        cube_friendly_bbox = np.vstack((start_coord, start_coord + (num_voxels * side_length))).T
        x_edges, y_edges, z_edges = [np.arange(minimum, maximum, side_length) for minimum, maximum in cube_friendly_bbox]
#         print([len(x_edges), len(y_edges), len(z_edges)])        
        
        # With this version, I am again missing a portion of it.
        offset_vectors = list()
        
        sorted_idx = mesh.vertices.T[0].argsort()
        x_verts = mesh.vertices[sorted_idx]
        splitter = x_verts.T[0].searchsorted(x_edges)
        x_split = np.split(x_verts, splitter)
        for i, x_block in enumerate(x_split):
            if len(x_block) > 0:
                sorted_idx = x_block.T[1].argsort()
                y_verts = x_block[sorted_idx]
                splitter = y_verts.T[1].searchsorted(y_edges)
                y_split = np.split(y_verts, splitter)
                if len(y_split) > 0:
                    for j, y_block in enumerate(y_split):
                        if len(y_block) > 0:
                            sorted_idx = y_block.T[2].argsort()
                            z_verts = y_block[sorted_idx]
                            splitter = z_verts.T[2].searchsorted(z_edges)
                            z_split = np.split(z_verts, splitter)
                            if len(z_split) > 0:
                                for k, z_block in enumerate(z_split):
                                    if len(z_block) > 0:
                                        offset_vectors.append((i, j, k))
        
        self.structure = np.array(offset_vectors) - 1
        
        return self.structure
    
        sorted_verts = self._sort_vertex_rows(mesh.vertices)

        splitter = sorted_verts[:,0].searchsorted(x_edges)
        x_split = np.split(sorted_verts, splitter)
        
        splitter = sorted_verts[:,1].searchsorted(y_edges)
        y_split = np.split(sorted_verts, splitter)
        
        splitter = sorted_verts[:,2].searchsorted(z_edges)
        z_split = np.split(sorted_verts, splitter)
        
        offset_vectors = list()
        
        # Potentially can now do a combinatorial thing between all of them. Maybe for each pair do a set intersection?
        # Well, I literally only care to see if there are any vertices at all at the intersection, then I'll return an offset vector for each voxel.
        for i, x_block in enumerate(x_split):
            # This way of doing combinatorial is very inefficient, should use a numpy.apply_across_axis or something once I verify it works first.
            # Or I just go the way of doing the searchsorted with this loop and just keep track of the voxel indices using the same i, j, k
            # btw pretty sure that i, j, k can be directly used as the offset vector indices.
            for j, y_block in enumerate(y_split):
                for k, z_block in enumerate(z_split):
                    if len(set.intersection(x_block, y_block, z_block)): offset_vectors.append((i, j, k))
        
        offset_vectors = np.array(offset_vectors)
        return offset_vectors
        
        # Tag the voxels that have vertices. Keep track of the structure offset indices. But should I also keep track of voxel ids? Probably not if
        # I do it where each voxel just holds the offset.
        
        
        # Could I tag vertices for each voxel? How expensive would that be, but it would be very easy and efficient querying and manipulating afterwords.
        
        # Query number of vertices in large chunks and then split down into the medium then 1-1 voxel resolution.
        
        # To get the indices of the vertex blocks with vertices, do ( BUT this doesn't actually work, but you get the idea )
        # summed = vertex_blocks.sum(axis=whatever_works)
        # idx = np.where(summed>0)
        
        # Will also need to have a voxel-level "bbox" where it re-corners (moves it to the topmost-leftmost corner, or some other corner) the starting
        # voxel structure centroid (or corner coordinate, this decision affects from where the offset is calculated) to the location that agrees with the
        # existing offset distance settings and then it should be able to easily add/subtract the offset change from the offset indices stored in the voxel
        # offset vectors.

In [5]:
mesh = Voxel(fetched_mesh['vertices'])

In [6]:
%%timeit

thing = mesh.cube_voxelize(5000) #2500
# print(len(thing))

192 ms ± 8.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
mesh.plot_structure()

In [122]:
eh = thing[0]

In [125]:
eh.tolist()

[[213431.765625, 222495.484375, 43725.24609375],
 [213457.109375, 222541.9375, 43622.21875],
 [213474.0625, 222514.5625, 43789.96875],
 [213486.984375, 222420.21875, 43624.40625],
 [213521.0625, 222389.25, 43453.3125],
 [213521.984375, 222590.96875, 43493.3125],
 [213523.0625, 222672.109375, 43643.9765625],
 [213542.359375, 222672.75, 43564.67578125],
 [213551.765625, 222555.453125, 43788.84375],
 [213560.28125, 222302.46875, 43445.125],
 [213618.578125, 222656.890625, 43625.8125],
 [213634.875, 222271.265625, 43530.25],
 [213642.078125, 222308.78125, 43742.546875],
 [213643.65625, 222247.28125, 43665.9375],
 [213646.40625, 222621.5625, 43336.34375],
 [213664.25, 222500.453125, 43328.90625],
 [213666.46875, 222735.578125, 43514.015625],
 [213709.5625, 222391.96875, 43772.39453125],
 [213737.1875, 222392.0, 43408.0625],
 [213777.96875, 222590.015625, 43335.8046875],
 [213787.171875, 222511.015625, 43612.6875],
 [213828.53125, 222412.578125, 43628.5],
 [213829.75, 222442.09375, 43340.285

In [124]:
set()

TypeError: unhashable type: 'list'

In [105]:
sum([len(thin) for thin in thing])

338990

In [ ]:
# @schema if schema = dj.schema('microns_ta3p100')
class VoxelConfig(dj.Lookup):
    definition = """
    side_length   : decimal(10, 5)
    ---
    # side_length : float # hmm, I wanted the side length to be something I can use. Maybe I should force them to be integers or decimal? Yeah decimal
                        # would actually pretty helpful.
    """
    
class Voxels(dj.Computed):
    definition = """
    -> ta3p100.Mesh
    -> VoxelConfig
    """
    
    key_source = ta3p100.Mesh * VoxelConfig
    
    class Structure(dj.Part):

In [ ]:
# outdated

sorted_idx = nonisolated_vertices.T[0].argsort()
x_verts = nonisolated_vertices[sorted_idx]
splitter = x_verts.T[0].searchsorted(x_edges)
x_split = np.split(x_verts, splitter)
for x_block in x_split:
    if len(x_block) > 0:
        sorted_idx = x_block.T[1].argsort()
        y_verts = x_block[sorted_idx]
        splitter = y_verts.T[1].searchsorted(y_edges)
        y_split = np.split(y_verts, splitter)
        if len(y_split) > 0:
            for y_block in y_split:
                if len(y_block) > 0:
                    sorted_idx = y_block.T[2].argsort()
                    z_verts = y_block[sorted_idx]
                    splitter = z_verts.T[2].searchsorted(z_edges)
                    z_split = np.split(z_verts, splitter)
                    if len(z_split) > 0:
                        for z_block in z_split:
                            if len(z_block) > 0:
                                bboxes.append(self.get_bbox(z_block))